In [10]:
import os
from dotenv import load_dotenv
load_dotenv()
api_key=os.getenv("pinecone_api_key")

In [5]:
from langchain_community.retrievers import PineconeHybridSearchRetriever

In [6]:
from pinecone import Pinecone,ServerlessSpec
index_name="hybrid-search-langchain-pinecone"
pc=Pinecone(api_key=api_key)

In [7]:
#create the index
if index_name not in pc.list_indexes().names():
    pc.create_index(
        name=index_name,
        dimension=384, # dimension of dense vector
        metric='dotproduct', #sparse value supported only for dotproduct
        spec=ServerlessSpec(cloud='aws',region='us-east-1')    
    )

In [8]:
index=pc.Index(index_name)
index

In [11]:
from langchain_huggingface import HuggingFaceEmbeddings

os.environ["HUGGINGFACE_TOKEN"]=os.getenv("HUGGINGFACE_TOKEN")

embeddings=HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
embeddings

HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, query_encode_kwargs={}, multi_process=False, show_progress=False)

In [12]:
from pinecone_text.sparse import BM25Encoder

bm25_encoder=BM25Encoder().default()
bm25_encoder

In [13]:
sentences=[
    "pinecone is a vector database",
    "it stores embeddings (vectors)",
    "and helps you to search them super fast"
]

#tf idf values on these sentence
bm25_encoder.fit(sentences)

#store the values to a json file
bm25_encoder.dump("bm25_values.json")

#load to BM25Encoder object
bm25_encoder=BM25Encoder().load("bm25_values.json")

100%|██████████| 3/3 [00:00<00:00, 15.08it/s]


In [14]:
retriever=PineconeHybridSearchRetriever(embeddings=embeddings,sparse_encoder=bm25_encoder,index=index)

In [15]:
retriever.add_texts([
    "pinecone is a vector database",
    "it stores embeddings (vectors)",
    "and helps you to search them super fast"
])

100%|██████████| 1/1 [00:02<00:00,  2.95s/it]


In [16]:
retriever.invoke("what is pinecone?")

[Document(metadata={'score': 0.562454224}, page_content='pinecone is a vector database'),
 Document(metadata={'score': 0.0542149544}, page_content='and helps you to search them super fast'),
 Document(metadata={'score': 0.0913567543}, page_content='it stores embeddings (vectors)')]